In [1]:
import numpy as np
with open('./input.txt','r') as file:
    hex_in = file.readline()
    

def hex2bin(hex_in):
    binary = ''
    for i in hex_in:
        if i == '0':
            binary += '0000'
        elif i == '1':
            binary += '0001'
        elif i == '2':
            binary += '0010'
        elif i == '3':
            binary += '0011'
        elif i == '4':
            binary += '0100'
        elif i == '5':
            binary += '0101'
        elif i == '6':
            binary += '0110'
        elif i == '7':
            binary += '0111'
        elif i == '8':
            binary += '1000'
        elif i == '9':
            binary += '1001'
        elif i == 'A':
            binary += '1010'
        elif i == 'B':
            binary += '1011'
        elif i == 'C':
            binary += '1100'
        elif i == 'D':
            binary += '1101'
        elif i == 'E':
            binary += '1110'
        elif i == 'F':
            binary += '1111'   
    return binary


def read_DEC(ptr, LEN,raw):
    temp = raw[ptr:ptr+LEN]
    number = 0
    for idx, t in enumerate(temp[::-1]):
        number += int(t) * 2**idx
    ptr += LEN

    return number, ptr

def read_BIN(ptr, LEN,raw):
    temp = raw[ptr:ptr+LEN]
    number = []    
    for t in temp:
        number.append(int(t))
    ptr += LEN
    return number, ptr

def clear_zeros(ptr):
    while (ptr % 4) != 1:
        ptr += 1
    return ptr

In [2]:
def parse_packet(ptr,raw, v_tot):
    v, ptr = read_DEC(ptr,V_LEN,raw)
    t, ptr = read_DEC(ptr,T_LEN,raw)
    
    packet = {
            'v':v,
            't':t
    }    
    v_tot += v
    strip = False
    
    if t == 4:
        go_stop, ptr = read_DEC(ptr,1,raw)
        bin_arr = []
        while go_stop:   #read numbers while go_stop == 1            
            b,ptr = read_BIN(ptr,A_LEN,raw)
            bin_arr.append(b)
            go_stop, ptr = read_DEC(ptr,1,raw)
        b, ptr = read_BIN(ptr,A_LEN,raw)   #read the final go_stop == 0
        bin_arr.append(b)
        bin_arr = np.reshape(bin_arr,(-1))
        dec = 0
        for idx, b in enumerate(bin_arr[::-1]):
            dec += b * 2**idx
        packet['dec'] = dec

    else:
        i, ptr = read_DEC(ptr,1,raw)
        packet['child'] = []
        if i == 0:
            l, ptr = read_DEC(ptr,L1_LEN,raw)
            packet['l'] = l
            packet['i'] = i
            p_start = ptr
            
            temp_raw = raw[ptr:l+p_start]
            temp_ptr = 0
            while ptr < l+p_start:
                p, temp_ptr, v_tot = parse_packet(temp_ptr,temp_raw, v_tot)
                packet['child'].append(p)   
                ptr += temp_ptr
                temp_raw = raw[ptr:l+p_start]
                temp_ptr = 0    
            strip = True
        else:
            l, ptr = read_DEC(ptr,L2_LEN,raw)
            packet['children'] = l
            packet['i'] = i
            p_start = ptr
            temp_raw = raw[ptr:]

            temp_ptr = 0   
            for idx in range(l):                 
                p, temp_ptr, v_tot = parse_packet(temp_ptr,temp_raw, v_tot)
                packet['child'].append(p) 
                ptr += temp_ptr
                
                temp_raw = raw[ptr:]
                temp_ptr = 0  
            strip = True
    if strip:
        while ptr < len(raw) and raw[ptr] == 0:
            ptr += 1
    return packet,ptr, v_tot

In [3]:


V_LEN = 3
T_LEN = 3
A_LEN = 4
L1_LEN = 15
L2_LEN = 11

raw = hex2bin(hex_in)

ptr = 0
v_tot = 0


packet,ptr, v_tot = parse_packet(ptr, raw, v_tot)
v_tot

860

In [4]:

def reverse_polish(packet, stack):
    if packet['t'] == 0:
        stack.append('+')
    elif packet['t'] == 1:
        stack.append('*')
    elif packet['t'] == 2:
        stack.append('min')    
    elif packet['t'] == 3:
        stack.append('max')    
    elif packet['t'] == 4:
        stack.append(packet['dec'])        
    elif packet['t'] == 5:
        stack.append('>')    
    elif packet['t'] == 6:
        stack.append('<')    
    elif packet['t'] == 7:
        stack.append('==')
    
    s_id = len(stack)
    
    if 'child' in packet.keys():  
        stack.append('(')
        for c in packet['child']:
            stack = reverse_polish(c, stack)
        stack.append(')')        
        
    return stack

stack = reverse_polish(packet,[])
while len(stack)>2:

    ptr = 0
    while stack[ptr] != ')':
        ptr += 1
    end_ptr = ptr
    while stack[ptr] != '(':
        ptr -= 1
    start_ptr = ptr

    operation = stack[start_ptr - 1]
    if operation == '+':
        result = np.sum(stack[start_ptr+1:end_ptr])
    elif operation == '*':
        result = np.prod(stack[start_ptr+1:end_ptr])
    elif operation == 'min':
        result = np.min(stack[start_ptr+1:end_ptr])    
    elif operation == 'max':
        result = np.max(stack[start_ptr+1:end_ptr])
    elif operation == '>':
        result = np.where(stack[start_ptr+1] > stack[start_ptr+2],1,0)
    elif operation == '<':
        result = np.where(stack[start_ptr+1] < stack[start_ptr+2],1,0)
    elif operation == '==':
        result = np.where(stack[start_ptr+1] == stack[start_ptr+2],1,0)
    
    stack[start_ptr-1] = result
    for i in range(end_ptr,start_ptr-1,-1):
        _ = stack.pop(i)
stack

[470949537659]